In [ ]:
import torch
print('GPU 可用：', torch.cuda.is_available(), '，设备名：', torch.cuda.get_device_name(0) if torch.cuda.is_available() else None)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


GPU 可用： False ，设备名： None


In [ ]:
# 导入embedding和stock label文件
import pandas as pd
from google.colab import drive
import torch
drive.mount('/content/drive')

# 2. 指定 .pt 文件路径
pt_path = '/content/drive/My Drive/545 Group/datasets/embeddings_split0.pt'

# 3. 用 torch.load 读取
embeddings = torch.load(pt_path, weights_only=False)

# 查看类型和内容
print(type(embeddings))
try:
    print(embeddings.shape)
except:
    # 如果是 dict 或 list
    print(embeddings.keys() if isinstance(embeddings, dict) else len(embeddings))

# 2. 读入标签
stock_path = '/content/drive/My Drive/545 Group/orgi/labeled_stock.csv'

df_labels = pd.read_csv(stock_path, parse_dates=["Date"])

# 把标签表里的 Date 列也格式化成 "YYYY-MM-DD" 字符串，以便跟 result['date'] 对齐
df_labels["date"] = df_labels["Date"].dt.strftime("%Y-%m-%d")
df_labels = df_labels[["date", "label"]]

Mounted at /content/drive
<class 'list'>
1943232


In [ ]:
# 把embedding转成dataframe

import pandas as pd
import numpy as np

# 构造 DataFrame
df1 = pd.DataFrame({
    'date':      [sample['date'] for sample in embeddings],
    'embedding': [np.asarray(sample['embedding']).tolist() for sample in embeddings]
})

print(df1.shape)  # (194323, 2)
display(df1.head())


(1943232, 2)


,date,embedding
0,2023-12-16 23:00:00 UTC,"[-0.90576171875, -0.751953125, -0.9228515625, ..."
1,2023-12-12 00:00:00 UTC,"[-0.88232421875, -0.423583984375, -0.860839843..."
2,2023-12-12 00:00:00 UTC,"[-0.76904296875, -0.422119140625, -0.766113281..."
3,2023-12-07 00:00:00 UTC,"[-0.79150390625, -0.2430419921875, -0.08081054..."
4,2023-12-07 00:00:00 UTC,"[-0.371826171875, -0.320068359375, -0.9609375,..."


In [ ]:
# 把date 去掉小时分钟和UTC，再加一
df = df1.copy()

# 1) 解析成 datetime 并只保留日期部分
#    去掉末尾 " UTC"，再 parse，最后保留 .dt.date
df['date'] = df1['date'].str[:10]

# 1) 先转成 datetime 类型
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')

# 2) 加一天
df['date'] = df['date'] + pd.Timedelta(days=1)

# 3) 如果你还想把它变回字符串
df['date'] = df['date'].dt.strftime('%Y-%m-%d')

# 2) 把 embedding 列转换成 ndarray
df['emb_arr'] = df['embedding'].apply(np.asarray)

print(df.shape)
display(df.head())

(1943232, 3)


,date,embedding,emb_arr
0,2023-12-17,"[-0.90576171875, -0.751953125, -0.9228515625, ...","[-0.90576171875, -0.751953125, -0.9228515625, ..."
1,2023-12-13,"[-0.88232421875, -0.423583984375, -0.860839843...","[-0.88232421875, -0.423583984375, -0.860839843..."
2,2023-12-13,"[-0.76904296875, -0.422119140625, -0.766113281...","[-0.76904296875, -0.422119140625, -0.766113281..."
3,2023-12-08,"[-0.79150390625, -0.2430419921875, -0.08081054...","[-0.79150390625, -0.2430419921875, -0.08081054..."
4,2023-12-08,"[-0.371826171875, -0.320068359375, -0.9609375,...","[-0.371826171875, -0.320068359375, -0.9609375,..."


In [ ]:
# 查看重复日期数量

# 总行数
total = len(df)

# 不同日期的个数
unique_dates = df['date'].nunique()

# 重复的日期总数
dup_count = total - unique_dates
print(f'总行数: {total}，唯一日期数: {unique_dates}，重复日期条目数: {dup_count}')

# 如果你想看每个日期出现的次数：
counts = df['date'].value_counts()
print(counts)

# 或者直接计算有多少行是重复（只保留第一次出现为非重复）：
dup_rows = df['date'].duplicated().sum()
print(f'使用duplicated()算出的重复行数: {dup_rows}')


总行数: 1943232，唯一日期数: 5089，重复日期条目数: 1938143
date
2023-12-12    47950
2023-12-17    43651
2023-12-13    43420
2023-12-14    26925
2023-12-08    25769
              ...  
2012-07-09        1
2009-06-07        1
2014-03-09        1
2012-12-16        1
2012-01-01        1
Name: count, Length: 5089, dtype: int64
使用duplicated()算出的重复行数: 1938143


In [ ]:
import torch
import numpy as np
import pandas as pd

# 1) 按 date 合并，保证每条新闻都有对应的当天标签
merged = pd.merge(df[['date','emb_arr']], df_labels, on='date', how='inner')
# merged 有 ['date','emb_arr','label']

# 2) 分组构造 sequences 和 labels
sequences = []
labels    = []

for date, group in merged.groupby('date'):
    # group['emb_arr'] 是当天多条新闻的 np.ndarray list
    # 转成形状 (N_i, D) 的 Tensor
    seq = torch.stack([torch.tensor(v, dtype=torch.float32)
                       for v in group['emb_arr'].values], dim=0)
    sequences.append(seq)
    # 当天只有一个标签
    labels.append(int(group['label'].iloc[0]))




In [ ]:
merged.head()

,date,emb_arr,label
0,2023-12-13,"[-0.88232421875, -0.423583984375, -0.860839843...",2
1,2023-12-13,"[-0.76904296875, -0.422119140625, -0.766113281...",2
2,2023-12-08,"[-0.79150390625, -0.2430419921875, -0.08081054...",2
3,2023-12-08,"[-0.371826171875, -0.320068359375, -0.9609375,...",2
4,2023-12-06,"[-0.89013671875, -0.4248046875, -0.6533203125,...",2


In [ ]:
import random
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm

# ——————————————————————————————————————————————
# 1) 准备序列和标签（和你原来一样）
# ——————————————————————————————————————————————
sequences, labels = [], []
for date, group in merged.groupby('date'):
    seq = torch.stack([
        torch.tensor(v, dtype=torch.float32)
        for v in group['emb_arr'].values
    ], dim=0)  # (N_i, D)
    sequences.append(seq)
    labels.append(int(group['label'].iloc[0]))

# one-hot 编码所有真实标签，用于训练时的“历史”拼接
num_classes = len(set(labels))
onehot = np.eye(num_classes)[labels]  # (T, C)

# 构造带历史信号的训练样本列表
K = 3
seqs2, hist_feats, labs2 = [], [], []
for i in range(len(sequences)):
    if i < K:
        hist = np.zeros((K, num_classes), dtype=float)
    else:
        hist = onehot[i-K:i]          # shape (K, C)
    seqs2.append(sequences[i])
    hist_feats.append(torch.tensor(hist.flatten(), dtype=torch.float32))
    labs2.append(labels[i])


In [ ]:

# ——————————————————————————————————————————————
# 2) Dataset + collate_fn (返回 x, mask, hist_true, y)
# ——————————————————————————————————————————————
class DateNewsWithHist(Dataset):
    def __init__(self, seqs, hists, labs):
        self.seqs  = seqs
        self.hists = hists
        self.labs  = labs
    def __len__(self):
        return len(self.labs)
    def __getitem__(self, i):
        return self.seqs[i], self.hists[i], self.labs[i]

MAX_LEN = 512
def sample_seq(s, N):
    L = s.size(0)
    if L <= N: return s
    idx = sorted(random.sample(range(L), N))
    return s[idx]

def collate_fn(batch):
    seqs, hists, labs = zip(*batch)
    # 随机采样 / 截断
    seqs = [sample_seq(s, MAX_LEN) for s in seqs]
    padded = pad_sequence(seqs, batch_first=True, padding_value=0.0)
    B, S, D = padded.shape
    mask = torch.zeros(B, S, dtype=torch.bool)
    for i,s in enumerate(seqs):
        mask[i, :s.size(0)] = 1
    hists = torch.stack(hists, dim=0)            # (B, K*C)
    labs  = torch.tensor(labs, dtype=torch.long) # (B,)
    return padded, mask, hists, labs

# ——————————————————————————————————————————————
# 3) 划分 + DataLoader
# ——————————————————————————————————————————————
seq_tr, seq_va, hist_tr, hist_va, lab_tr, lab_va = train_test_split(
    seqs2, hist_feats, labs2,
    test_size=0.2, random_state=42, stratify=labs2
)
train_ds = DateNewsWithHist(seq_tr, hist_tr, lab_tr)
val_ds   = DateNewsWithHist(seq_va, hist_va, lab_va)
train_loader = DataLoader(train_ds, batch_size=16, shuffle=True,  collate_fn=collate_fn)
val_loader   = DataLoader(val_ds,   batch_size=16, shuffle=False, collate_fn=collate_fn)

test_ds     = val_ds
test_loader = DataLoader(
    test_ds, batch_size=1, shuffle=False, collate_fn=collate_fn
)


In [ ]:

# ——————————————————————————————————————————————
# 4) 定义带历史信号的 Transformer 模型
# ——————————————————————————————————————————————
class TransformerWithHist(nn.Module):
    def __init__(self, embed_dim, num_classes, hist_dim, **kw):
        super().__init__()
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embed_dim,
            nhead=kw.pop('nhead'),
            dim_feedforward=kw.pop('dim_feedforward'),
            dropout=kw.pop('dropout'),
            batch_first=True
        )
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=kw.pop('num_layers'))
        self.classifier = nn.Sequential(
            nn.Linear(embed_dim + hist_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(256, num_classes)
        )

    def forward(self, x, mask, hist_feat):
        # x: (B,S,E), mask: (B,S), hist_feat: (B, hist_dim)
        key_mask = ~mask
        x_enc = self.encoder(x, src_key_padding_mask=key_mask)  # (B,S,E)
        x_mean = x_enc.mean(dim=1)                              # (B,E)
        z      = torch.cat([x_mean, hist_feat], dim=1)          # (B, E+hist_dim)
        return self.classifier(z)

# ——————————————————————————————————————————————
# 5) 初始化 + 训练/验证循环
# ——————————————————————————————————————————————
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
embed_dim   = seq_tr[0].shape[1]
hist_dim    = K * num_classes
model = TransformerWithHist(
    embed_dim=embed_dim, num_classes=num_classes, hist_dim=hist_dim,
    nhead=8, num_layers=2, dim_feedforward=512, dropout=0.1
).to(device)
opt = torch.optim.AdamW(model.parameters(), lr=2e-4)
criterion = nn.CrossEntropyLoss()

for epoch in range(1, 6):
    # —— 训练 ——
    model.train()
    tl = 0
    for x, mask, h, y in tqdm(train_loader, desc=f"Train {epoch}"):
        x, mask, h, y = x.to(device), mask.to(device), h.to(device), y.to(device)
        opt.zero_grad()
        logits = model(x, mask, h)
        loss   = criterion(logits, y)
        loss.backward()
        opt.step()
        tl += loss.item()
    tl /= len(train_loader)

    # —— 验证 ——
    model.eval()
    cl, corr, tot = 0, 0, 0
    with torch.no_grad():
        for x, mask, h, y in val_loader:
            x, mask, h, y = x.to(device), mask.to(device), h.to(device), y.to(device)
            logits = model(x, mask, h)
            cl += criterion(logits, y).item()
            preds = logits.argmax(dim=1)
            corr += (preds==y).sum().item()
            tot  += y.size(0)
    print(f"[{epoch}] train_loss={tl:.4f}  val_loss={cl/len(val_loader):.4f}  val_acc={corr/tot:.4f}")

# # ——————————————————————————————————————————————
# # 6) 推理：用滑动窗口维护“预测信号”作为历史
# # ——————————————————————————————————————————————
# history = [0]*K
# preds_all = []
# for x, mask, _, _ in test_loader:  # test_loader 同样返回 4 个，但我们忽略第3个（hist_true）
#     x, mask = x.to(device), mask.to(device)
#     # 构造 hist_pred one-hot
#     hist_feat = torch.eye(num_classes, device=device)[history].reshape(1, -1)
#     hist_feat = hist_feat.expand(x.size(0), -1)         # (B, K*C)
#     logits = model(x, mask, hist_feat)
#     batch_preds = logits.argmax(dim=1).cpu().tolist()
#     preds_all.extend(batch_preds)
#     # 更新 history 滑动窗口
#     history = history[len(batch_preds):] + batch_preds[:len(history)]

# # preds_all 即为测试集的预测结果
# from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# # 假设 labels_test 是一个 Python 列表或 1D np.array，和 preds_all 等长
# # e.g.
# # labels_test = labs2[K:][ split_index: ]   # 对应你 test_loader 使用的那一段数据

# labels_test = labs2[K:]

# acc = accuracy_score(labels_test, preds_all)
# print(f"Test Accuracy: {acc:.4f}")

# print("Classification Report:")
# print(classification_report(labels_test, preds_all, digits=4))

# print("Confusion Matrix:")
# print(confusion_matrix(labels_test, preds_all))



Train 1: 100%|██████████| 174/174 [01:35<00:00,  1.82it/s]


[1] train_loss=0.7630  val_loss=0.7887  val_acc=0.7871


Train 2: 100%|██████████| 174/174 [01:34<00:00,  1.84it/s]


[2] train_loss=0.6711  val_loss=0.6966  val_acc=0.8029


Train 3: 100%|██████████| 174/174 [01:43<00:00,  1.68it/s]


[3] train_loss=0.5960  val_loss=0.6333  val_acc=0.8086


Train 4: 100%|██████████| 174/174 [01:53<00:00,  1.54it/s]


[4] train_loss=0.5375  val_loss=0.5845  val_acc=0.8489


Train 5: 100%|██████████| 174/174 [01:58<00:00,  1.47it/s]


[5] train_loss=0.4987  val_loss=0.5486  val_acc=0.8432
